## BiDAF (Bi-Directional Attention Flow)

## Goal: Maximize the EM and F1 score to the SQuAD dataset by finetuning the existing BiDAF hyperparameter settings.


First, we will test learning rate and optimizer individually as they are relatively isolated from other variables but very close to each other.

In [1]:
# we use these flags to make the script portable for training
running_in_colab=False
running_in_local=True
load_train_epochs=3 # how many epochs you have already trained

While the original batch size was 16, we were forced to decrease it to 8 due to the limited amount of computation resources. A higher batch size would cause the runtime to be out of memory, therefore making the training impossible. Moreover, a smaller batch size would also help the model to achieve a better performance when finetuning.

In [2]:
BATCH_SIZE=8

In [3]:
optim_configuration=0

In [4]:
optimizer_config_list=[]
# configuration 0, all default
optimizer_config_list.append(("AdaDelta",1,"AdaDelta_default"))
# configuration 1
optimizer_config_list.append(("Adam",.001,"Adam_default"))
# configuration 2
optimizer_config_list.append(("SGD",.01,"SGD_default"))

In [5]:
OPTIMIZER, LEARNING_RATE, model_name = optimizer_config_list[optim_configuration]

In [63]:
lr_configuration=2

In [64]:
lr_config_list=[]
# configuration 0, all default
lr_config_list.append(("AdaDelta",1,"AdaDelta_default"))
# configuration 1
lr_config_list.append(("AdaDelta",2,"AdaDelta_doubled"))
# configuration 2
lr_config_list.append(("AdaDelta",.5,"AdaDelta_halved"))

In [65]:
OPTIMIZER, LEARNING_RATE, model_name = lr_config_list[lr_configuration]

Now, we have determined that AdaDelta with learning rate of 0.5 works best for this model with a F1 score of over 55 and EM score of over 67 after 3 epochs. We will then move onto batch size, which is also very much separated from other parameters.

Note: model_name will be overwritten here. Please assume the optimizer and learning rate picked is the configuration with the best overall performance.

In [12]:
dropout_rate_configuration=0

In [13]:
dropout_config_list=[]
# configuration 0
dropout_config_list.append((.2,model_name))
# configuration 1
dropout_config_list.append((0.15,"dropout-.15"))
# configuration 2
dropout_config_list.append((0.25,"dropout-.25"))

In [14]:
DROPOUT_RATE, model_name=dropout_config_list[dropout_rate_configuration]

Note: A 0.15 dropout rate actually performed better under default AdaDelta learning rate, and was thus used in the experiments of the subsequent hyperparameters. However, the final model excluded such choice as the performance of it peaked at the second epoch instead of the last epoch as expected.  Hence, we decided that making the dropout rate 0.2 would be more favorable when we train 2 more epochs 

In [15]:
char_width_configuration=0

In [16]:
char_width_config_list=[]
# configuration 0
char_width_config_list.append((5,model_name))
# configuration 1
char_width_config_list.append((4,"charwidth4"))
# configuration 2
char_width_config_list.append((6,"charwidth6"))

In [17]:
CHARACTER_CHANNEL_WIDTH, model_name=char_width_config_list[char_width_configuration]

In [18]:
hidden_configuration=0

In [19]:
hidden_config_list=[]
# configuration 0
hidden_config_list.append((100,model_name))
# configuration 1
hidden_config_list.append((50,"hidden50"))
# configuration 2
hidden_config_list.append((200,"hidden200"))

In [20]:
HIDDEN_SIZE, model_name=hidden_config_list[hidden_configuration]

After testing all of the configurations above, the final model was decided (the configurations above). We decided to train it to 5 epochs to observe if it was improved. The state dictionary, loss, EM, F1 of each experiment can be found under [model_name].pth inside the results folder.

In [21]:
!pip install spacy

In [22]:
from torch import nn
import torch
import numpy as np
import pandas as pd
import pickle, time
import re, os, string, typing, gc, json
import torch.nn.functional as F
import spacy
from sklearn.model_selection import train_test_split
from collections import Counter
nlp = spacy.blank('en')
from preprocess import *
%load_ext autoreload
%autoreload 2

## Data Preprocessing

Please make sure you have downloaded the SQuAD dataset JSON files and stored them under /data before proceeding.

In [23]:
# load dataset json files

train_data = load_json('./data/squad_train.json')
valid_data = load_json('./data/squad_dev.json')

# parse the json structure to return the data as a list of dictionaries

train_list = parse_data(train_data)
valid_list = parse_data(valid_data)
print('--------------------------')

print('Train list len: ',len(train_list))
print('Valid list len: ',len(valid_list))

# converting the lists into dataframes

train_df = pd.DataFrame(train_list)
valid_df = pd.DataFrame(valid_list)

Length of data:  442
Data Keys:  dict_keys(['title', 'paragraphs'])
Title:  University_of_Notre_Dame
Length of data:  48
Data Keys:  dict_keys(['title', 'paragraphs'])
Title:  Super_Bowl_50
--------------------------
Train list len:  87599
Valid list len:  34726


In [24]:
train_df.head()

,id,context,question,label,answer
0,5733be284776f41900661182,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"[515, 541]",Saint Bernadette Soubirous
1,5733be284776f4190066117f,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"[188, 213]",a copper statue of Christ
2,5733be284776f41900661180,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"[279, 296]",the Main Building
3,5733be284776f41900661181,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,"[381, 420]",a Marian place of prayer and reflection
4,5733be284776f4190066117e,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,"[92, 126]",a golden statue of the Virgin Mary


In [25]:
def preprocess_df(df):
    
    def to_lower(text):
        return text.lower()

    df.context = df.context.apply(to_lower)
    df.question = df.question.apply(to_lower)
    df.answer = df.answer.apply(to_lower)

In [26]:
preprocess_df(train_df)
preprocess_df(valid_df)

In [27]:
# gather text to build vocabularies

%time vocab_text = gather_text_for_vocab([train_df, valid_df])
print("Number of sentences in dataset: ", len(vocab_text))

Wall time: 492 ms
Number of sentences in dataset:  118822


In [28]:
# build word and character-level vocabularies

%time word2idx, idx2word, word_vocab = build_word_vocab(vocab_text)
print("----------------------------------")
%time char2idx, char_vocab = build_char_vocab(vocab_text)

raw-vocab: 96774
vocab-length: 96776
word2idx-length: 96776
Wall time: 14 s
----------------------------------
raw-char-vocab: 1316
char-vocab-intersect: 202
char2idx-length: 204
Wall time: 2.67 s


In [29]:
# numericalize context and questions for training and validation set

%time train_df['context_ids'] = train_df.context.apply(context_to_ids, word2idx=word2idx)
%time valid_df['context_ids'] = valid_df.context.apply(context_to_ids, word2idx=word2idx)
%time train_df['question_ids'] = train_df.question.apply(question_to_ids, word2idx=word2idx)
%time valid_df['question_ids'] = valid_df.question.apply(question_to_ids, word2idx=word2idx)

Wall time: 18.2 s
Wall time: 7.32 s
Wall time: 2.97 s
Wall time: 1.05 s


In [30]:
# get indices with tokenization errors and drop those indices 

train_err = get_error_indices(train_df, idx2word)
valid_err = get_error_indices(valid_df, idx2word)

train_df.drop(train_err, inplace=True)
valid_df.drop(valid_err, inplace=True)

Number of error indices: 921
Number of error indices: 349


In [31]:
# get start and end positions of answers from the context
# this is basically the label for training QA models

train_label_idx = train_df.apply(index_answer, axis=1, idx2word=idx2word)
valid_label_idx = valid_df.apply(index_answer, axis=1, idx2word=idx2word)

train_df['label_idx'] = train_label_idx
valid_df['label_idx'] = valid_label_idx

In [32]:
# dump to pickle files

train_df.to_pickle('bidaftrain.pkl')
valid_df.to_pickle('bidafvalid.pkl')

with open('bidafw2id.pickle','wb') as handle:
    pickle.dump(word2idx, handle)

with open('bidafc2id.pickle','wb') as handle:
    pickle.dump(char2idx, handle)

In [33]:
# load data from pickle files


train_df = pd.read_pickle('bidaftrain.pkl')
valid_df = pd.read_pickle('bidafvalid.pkl')

with open('bidafw2id.pickle','rb') as handle:
    word2idx = pickle.load(handle)
with open('bidafc2id.pickle','rb') as handle:
    char2idx = pickle.load(handle)

idx2word = {v:k for k,v in word2idx.items()}

## Dataloader/Dataset

In [34]:
class SquadDataset:
    '''
    - Creates batches dynamically by padding to the length of largest example
      in a given batch.
    - Calulates character vectors for contexts and question.
    - Returns tensors for training.
    '''
    
    def __init__(self, data, batch_size):
        
        self.batch_size = batch_size
        data = [data[i:i+self.batch_size] for i in range(0, len(data), self.batch_size)]
        self.data = data
        
        
    def __len__(self):
        return len(self.data)
    
    def make_char_vector(self, max_sent_len, max_word_len, sentence):
        
        char_vec = torch.ones(max_sent_len, max_word_len).type(torch.LongTensor)
        
        for i, word in enumerate(nlp(sentence, disable=['parser','tagger','ner',"lemmatizer"])):
            for j, ch in enumerate(word.text):
                char_vec[i][j] = char2idx.get(ch, 0)
        
        return char_vec    
    
    def get_span(self, text):
        
        text = nlp(text, disable=['parser','tagger','ner'])
        span = [(w.idx, w.idx+len(w.text)) for w in text]

        return span

    def __iter__(self):
        '''
        Creates batches of data and yields them.
        
        Each yield comprises of:
        :padded_context: padded tensor of contexts for each batch 
        :padded_question: padded tensor of questions for each batch 
        :char_ctx & ques_ctx: character-level ids for context and question
        :label: start and end index wrt context_ids
        :context_text,answer_text: used while validation to calculate metrics
        :ids: question_ids for evaluation
        
        '''
        
        for batch in self.data:
            
            spans = []
            ctx_text = []
            answer_text = []
            
            for ctx in batch.context:
                ctx_text.append(ctx)
                spans.append(self.get_span(ctx))
            
            for ans in batch.answer:
                answer_text.append(ans)
                
            
            max_context_len = max([len(ctx) for ctx in batch.context_ids])
            padded_context = torch.LongTensor(len(batch), max_context_len).fill_(1)
            
            for i, ctx in enumerate(batch.context_ids):
                padded_context[i, :len(ctx)] = torch.LongTensor(ctx)
                
            max_word_ctx = 0
            for context in batch.context:
                for word in nlp(context, disable=['parser','tagger','ner']):
                    if len(word.text) > max_word_ctx:
                        max_word_ctx = len(word.text)
            
            char_ctx = torch.ones(len(batch), max_context_len, max_word_ctx).type(torch.LongTensor)
            for i, context in enumerate(batch.context):
                char_ctx[i] = self.make_char_vector(max_context_len, max_word_ctx, context)
            
            max_question_len = max([len(ques) for ques in batch.question_ids])
            padded_question = torch.LongTensor(len(batch), max_question_len).fill_(1)
            
            for i, ques in enumerate(batch.question_ids):
                padded_question[i, :len(ques)] = torch.LongTensor(ques)
                
            max_word_ques = 0
            for question in batch.question:
                for word in nlp(question, disable=['parser','tagger','ner']):
                    if len(word.text) > max_word_ques:
                        max_word_ques = len(word.text)
            
            char_ques = torch.ones(len(batch), max_question_len, max_word_ques).type(torch.LongTensor)
            for i, question in enumerate(batch.question):
                char_ques[i] = self.make_char_vector(max_question_len, max_word_ques, question)
            
            ids = list(batch.id)  
            label = torch.LongTensor(list(batch.label_idx))
            
            yield (padded_context, padded_question, char_ctx, char_ques, label, ctx_text, answer_text, ids)
            
            

In [35]:
train_dataset = SquadDataset(train_df, BATCH_SIZE)

In [36]:
valid_dataset = SquadDataset(valid_df, 16)

In [37]:
a = next(iter(train_dataset))

## BiDAF Model

## Word Embedding

In [38]:
import zipfile
import io

In [39]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

'wget' 不是內部或外部命令、可執行的程式或批次檔。


In [40]:
def get_glove_dict():
    '''
    Parses the glove word vectors text file and returns a dictionary with the words as
    keys and their respective pretrained word vectors as values.

    '''
    encoding = 'utf-8'
    glove_dict = {}
    archive = zipfile.ZipFile('glove.6B.zip', 'r')
    with archive.open("glove.6B.100d.txt", "r") as f:
        for line in io.TextIOWrapper(f, encoding):
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], "float32")
            glove_dict[word] = vector
            
    f.close()
    
    return glove_dict

In [41]:
def get_glove_dict_local_machine(): # assumes zip downloaded
    '''
    Parses the glove word vectors text file and returns a dictionary with the words as
    keys and their respective pretrained word vectors as values.

    '''
    glove_dict = {}
    with open(f"./glove.6B.{HIDDEN_SIZE}d.txt", "r", encoding="utf-8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], "float32")
            glove_dict[word] = vector
            
    f.close()
    
    return glove_dict

In [42]:
if running_in_local:
    glove_dict = get_glove_dict_local_machine()
else:
    glove_dict=get_glove_dict()

In [43]:
def create_weights_matrix(glove_dict):
    '''
    Creates a weight matrix of the words that are common in the GloVe vocab and
    the dataset's vocab. Initializes OOV words with a zero vector.
    '''
    weights_matrix = np.zeros((len(word_vocab), HIDDEN_SIZE))
    words_found = 0
    for i, word in enumerate(word_vocab):
        try:
            weights_matrix[i] = glove_dict[word]
            words_found += 1
        except:
            pass
        
    return weights_matrix, words_found


In [44]:
weights_matrix, words_found = create_weights_matrix(glove_dict)
print("Words found in the GloVe vocab: " ,words_found)

Words found in the GloVe vocab:  72687


In [45]:
# dump the weights to load in future

np.save('bidafglove_tv.npy', weights_matrix)

## Character Embedding

In [46]:
class CharacterEmbeddingLayer(nn.Module):
    
    def __init__(self, char_vocab_dim, char_emb_dim, num_output_channels, kernel_size):
        
        super().__init__()
        
        self.char_emb_dim = char_emb_dim
        
        self.char_embedding = nn.Embedding(char_vocab_dim, char_emb_dim, padding_idx=1)
        
        self.char_convolution = nn.Conv2d(in_channels=1, out_channels=num_output_channels, kernel_size=kernel_size)
        
        self.relu = nn.ReLU()
    
        self.dropout = nn.Dropout(DROPOUT_RATE)
                
    def forward(self, x):
        # x = [bs, seq_len, word_len]
        # returns : [batch_size, seq_len, num_output_channels]
        # the output can be thought of as another feature embedding of dim 100.
        
        batch_size = x.shape[0] 
        
        x = self.dropout(self.char_embedding(x))
        # x = [bs, seq_len, word_len, char_emb_dim]
        
        # following three operations manipulate x in such a way that
        # it closely resembles an image. this format is important before 
        # we perform convolution on the character embeddings.
        
        x = x.permute(0,1,3,2)
        # x = [bs, seq_len, char_emb_dim, word_len]
        
        x = x.view(-1, self.char_emb_dim, x.shape[3])
        # x = [bs*seq_len, char_emb_dim, word_len]
        
        x = x.unsqueeze(1)
        # x = [bs*seq_len, 1, char_emb_dim, word_len]
        
        # x is now in a format that can be accepted by a conv layer. 
        # think of the tensor above in terms of an image of dimension
        # (N, C_in, H_in, W_in).
        
        x = self.relu(self.char_convolution(x))
        # x = [bs*seq_len, out_channels, H_out, W_out]
        
        x = x.squeeze()
        # x = [bs*seq_len, out_channels, W_out]
             
        x = F.max_pool1d(x, x.shape[2]).squeeze()
        # x = [bs*seq_len, out_channels, 1] => [bs*seq_len, out_channels]
        
        x = x.view(batch_size, -1, x.shape[-1])
        # x = [bs, seq_len, out_channels]
        # x = [bs, seq_len, features] = [bs, seq_len, 100]
        
        
        return x        

## Highway Networks

In [47]:
class HighwayNetwork(nn.Module):
    
    def __init__(self, input_dim, num_layers=2):
        
        super().__init__()
        
        self.num_layers = num_layers
        
        self.flow_layer = nn.ModuleList([nn.Linear(input_dim, input_dim) for _ in range(num_layers)])
        self.gate_layer = nn.ModuleList([nn.Linear(input_dim, input_dim) for _ in range(num_layers)])
        
    def forward(self, x):
        
        for i in range(self.num_layers):
            
            flow_value = F.relu(self.flow_layer[i](x))
            gate_value = torch.sigmoid(self.gate_layer[i](x))
            
            x = gate_value * flow_value + (1-gate_value) * x
        
        return x

## Contextual Embedding

In [48]:
class ContextualEmbeddingLayer(nn.Module):
    
    def __init__(self, input_dim, hidden_dim):
        
        super().__init__()
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True, bidirectional=True)
        
        self.highway_net = HighwayNetwork(input_dim)
        
    def forward(self, x):
        # x = [bs, seq_len, input_dim] = [bs, seq_len, emb_dim*2]
        # the input is the concatenation of word and characeter embeddings
        # for the sequence.
        
        highway_out = self.highway_net(x)
        # highway_out = [bs, seq_len, input_dim]
        
        outputs, _ = self.lstm(highway_out)
        # outputs = [bs, seq_len, emb_dim*2]
        
        return outputs

## Attention Flow Layer

In [49]:
class BiDAF(nn.Module):
    
    def __init__(self, char_vocab_dim, emb_dim, char_emb_dim, num_output_channels, 
                 kernel_size, ctx_hidden_dim, device):
        '''
        char_vocab_dim = len(char2idx)
        emb_dim = 100
        char_emb_dim = 8
        num_output_chanels = 100
        kernel_size = (8,5)
        ctx_hidden_dim = 100
        '''
        super().__init__()
        
        self.device = device
        
        self.word_embedding = self.get_glove_embedding()
        
        self.character_embedding = CharacterEmbeddingLayer(char_vocab_dim, char_emb_dim, 
                                                      num_output_channels, kernel_size)
        
        self.contextual_embedding = ContextualEmbeddingLayer(emb_dim*2, ctx_hidden_dim)
        
        self.dropout = nn.Dropout()
        
        self.similarity_weight = nn.Linear(emb_dim*6, 1, bias=False)
        
        self.modeling_lstm = nn.LSTM(emb_dim*8, emb_dim, bidirectional=True, num_layers=2, batch_first=True, dropout=0.2)
        
        self.output_start = nn.Linear(emb_dim*10, 1, bias=False)
        
        self.output_end = nn.Linear(emb_dim*10, 1, bias=False)
        
        self.end_lstm = nn.LSTM(emb_dim*2, emb_dim, bidirectional=True, batch_first=True)
        
    
    def get_glove_embedding(self):
        
        weights_matrix = np.load('bidafglove_tv.npy')
        num_embeddings, embedding_dim = weights_matrix.shape
        embedding = nn.Embedding.from_pretrained(torch.FloatTensor(weights_matrix).to(self.device),freeze=True)

        return embedding
        
    def forward(self, ctx, ques, char_ctx, char_ques):
        # ctx = [bs, ctx_len]
        # ques = [bs, ques_len]
        # char_ctx = [bs, ctx_len, ctx_word_len]
        # char_ques = [bs, ques_len, ques_word_len]
        
        ctx_len = ctx.shape[1]
        
        ques_len = ques.shape[1]
        
        ## GET WORD AND CHARACTER EMBEDDINGS
        
        ctx_word_embed = self.word_embedding(ctx)
        # ctx_word_embed = [bs, ctx_len, emb_dim]
        
        ques_word_embed = self.word_embedding(ques)
        # ques_word_embed = [bs, ques_len, emb_dim]
        
        ctx_char_embed = self.character_embedding(char_ctx)
        # ctx_char_embed =  [bs, ctx_len, emb_dim]
        
        ques_char_embed = self.character_embedding(char_ques)
        # ques_char_embed = [bs, ques_len, emb_dim]
        
        ## CREATE CONTEXTUAL EMBEDDING
        
        ctx_contextual_inp = torch.cat([ctx_word_embed, ctx_char_embed],dim=2)
        # [bs, ctx_len, emb_dim*2]
        
        ques_contextual_inp = torch.cat([ques_word_embed, ques_char_embed],dim=2)
        # [bs, ques_len, emb_dim*2]
        
        ctx_contextual_emb = self.contextual_embedding(ctx_contextual_inp)
        # [bs, ctx_len, emb_dim*2]
        
        ques_contextual_emb = self.contextual_embedding(ques_contextual_inp)
        # [bs, ques_len, emb_dim*2]
        
        
        ## CREATE SIMILARITY MATRIX
        
        ctx_ = ctx_contextual_emb.unsqueeze(2).repeat(1,1,ques_len,1)
        # [bs, ctx_len, 1, emb_dim*2] => [bs, ctx_len, ques_len, emb_dim*2]
        
        ques_ = ques_contextual_emb.unsqueeze(1).repeat(1,ctx_len,1,1)
        # [bs, 1, ques_len, emb_dim*2] => [bs, ctx_len, ques_len, emb_dim*2]
        
        elementwise_prod = torch.mul(ctx_, ques_)
        # [bs, ctx_len, ques_len, emb_dim*2]
        
        alpha = torch.cat([ctx_, ques_, elementwise_prod], dim=3)
        # [bs, ctx_len, ques_len, emb_dim*6]
        
        similarity_matrix = self.similarity_weight(alpha).view(-1, ctx_len, ques_len)
        # [bs, ctx_len, ques_len]
        
        
        ## CALCULATE CONTEXT2QUERY ATTENTION
        
        a = F.softmax(similarity_matrix, dim=-1)
        # [bs, ctx_len, ques_len]
        
        c2q = torch.bmm(a, ques_contextual_emb)
        # [bs] ([ctx_len, ques_len] X [ques_len, emb_dim*2]) => [bs, ctx_len, emb_dim*2]
        
        
        ## CALCULATE QUERY2CONTEXT ATTENTION
        
        b = F.softmax(torch.max(similarity_matrix,2)[0], dim=-1)
        # [bs, ctx_len]
        
        b = b.unsqueeze(1)
        # [bs, 1, ctx_len]
        
        q2c = torch.bmm(b, ctx_contextual_emb)
        # [bs] ([bs, 1, ctx_len] X [bs, ctx_len, emb_dim*2]) => [bs, 1, emb_dim*2]
        
        q2c = q2c.repeat(1, ctx_len, 1)
        # [bs, ctx_len, emb_dim*2]
        
        ## QUERY AWARE REPRESENTATION
        
        G = torch.cat([ctx_contextual_emb, c2q, 
                       torch.mul(ctx_contextual_emb,c2q), 
                       torch.mul(ctx_contextual_emb, q2c)], dim=2)
        
        # [bs, ctx_len, emb_dim*8]
        
        
        ## MODELING LAYER
        
        M, _ = self.modeling_lstm(G)
        # [bs, ctx_len, emb_dim*2]
        
        ## OUTPUT LAYER
        
        M2, _ = self.end_lstm(M)
        
        # START PREDICTION
        
        p1 = self.output_start(torch.cat([G,M], dim=2))
        # [bs, ctx_len, 1]
        
        p1 = p1.squeeze()
        # [bs, ctx_len]
        
        #p1 = F.softmax(p1, dim=-1)
        
        # END PREDICTION
        
        p2 = self.output_end(torch.cat([G, M2], dim=2)).squeeze()
        # [bs, ctx_len, 1] => [bs, ctx_len]
        
        #p2 = F.softmax(p2, dim=-1)
        
        
        return p1, p2
    

## Training

In [50]:
CHAR_VOCAB_DIM = len(char2idx)
EMB_DIM = HIDDEN_SIZE # need to match up with hidden size
CHAR_EMB_DIM = 8
NUM_OUTPUT_CHANNELS = HIDDEN_SIZE # need to match up with hidden size
KERNEL_SIZE = (8,CHARACTER_CHANNEL_WIDTH)
device = torch.device('cuda')

model = BiDAF(CHAR_VOCAB_DIM, 
              EMB_DIM, 
              CHAR_EMB_DIM, 
              NUM_OUTPUT_CHANNELS, 
              KERNEL_SIZE, 
              HIDDEN_SIZE, 
              device).to(device)

In [51]:
import torch.optim as optim
from torch.autograd import Variable

In [52]:
if OPTIMIZER=="AdaDelta":
    optimizer = optim.Adadelta(model.parameters(), lr=LEARNING_RATE)
elif OPTIMIZER=="Adam":
    optimizer=optim.Adam(model.parameters(), lr=LEARNING_RATE)
elif OPTIMIZER=="SGD":
    optimizer=optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9, weight_decay=5e-4) # momentum added
else:
    raise Exception("Unrecognized optimizer chosen")

In [53]:
import gc

def train(model, train_dataset):
    print("Starting training ........")
   
    train_loss = 0.
    batch_count = 0
    model.train()
    for batch in train_dataset:
        gc.collect()
        torch.cuda.empty_cache()
        optimizer.zero_grad()
    
        if batch_count % 500 == 0:
            print(f"Starting batch: {batch_count}")
        batch_count += 1
        
        context, question, char_ctx, char_ques, label, ctx_text, ans, ids = batch

        context, question, char_ctx, char_ques, label = context.to(device), question.to(device),\
                                   char_ctx.to(device), char_ques.to(device), label.to(device)


        preds = model(context, question, char_ctx, char_ques)

        start_pred, end_pred = preds

        s_idx, e_idx = label[:,0], label[:,1]

        loss = F.cross_entropy(start_pred, s_idx) + F.cross_entropy(end_pred, e_idx)

        loss.backward()

        optimizer.step()

        # this prevented memory overflow
        del context, question, char_ctx, char_ques, label, ctx_text, ans, ids, preds, start_pred, end_pred,s_idx, e_idx

        train_loss += loss.item()

    return train_loss/len(train_dataset)

In [54]:
def valid(model, valid_dataset):
    
    print("Starting validation .........")
   
    valid_loss = 0.

    batch_count = 0
    
    f1, em = 0., 0.
    
    model.eval()
        
   
    predictions = {}
    
    for batch in valid_dataset:

        if batch_count % 500 == 0:
            print(f"Starting batch {batch_count}")
        batch_count += 1

        context, question, char_ctx, char_ques, label, ctx, answers, ids = batch

        context, question, char_ctx, char_ques, label = context.to(device), question.to(device),\
                                   char_ctx.to(device), char_ques.to(device), label.to(device)
        
       

        
        with torch.no_grad():
            
            s_idx, e_idx = label[:,0], label[:,1]

            preds = model(context, question, char_ctx, char_ques)

            p1, p2 = preds

            
            loss = F.cross_entropy(p1, s_idx) + F.cross_entropy(p2, e_idx)

            valid_loss += loss.item()

            batch_size, c_len = p1.size()
            ls = nn.LogSoftmax(dim=1)
            mask = (torch.ones(c_len, c_len) * float('-inf')).to(device).tril(-1).unsqueeze(0).expand(batch_size, -1, -1)
            score = (ls(p1).unsqueeze(2) + ls(p2).unsqueeze(1)) + mask
            score, s_idx = score.max(dim=1)
            score, e_idx = score.max(dim=1)
            s_idx = torch.gather(s_idx, 1, e_idx.view(-1, 1)).squeeze()
            
           
            for i in range(batch_size):
                id = ids[i]
                pred = context[i][s_idx[i]:e_idx[i]+1]
                pred = ' '.join([idx2word[idx.item()] for idx in pred])
                predictions[id] = pred
            

    
    em, f1 = evaluate(predictions)
    return valid_loss/len(valid_dataset), em, f1

In [55]:
def evaluate(predictions):
    '''
    Gets a dictionary of predictions with question_id as key
    and prediction as value. The validation dataset has multiple 
    answers for a single question. Hence we compare our prediction
    with all the answers and choose the one that gives us
    the maximum metric (em or f1). 
    This method first parses the JSON file, gets all the answers
    for a given id and then passes the list of answers and the 
    predictions to calculate em, f1.
    
    
    :param dict predictions
    Returns
    : exact_match: 1 if the prediction and ground truth 
      match exactly, 0 otherwise.
    : f1_score: 
    '''
    with open('./data/squad_dev.json','r',encoding='utf-8') as f:
        dataset = json.load(f)
        
    dataset = dataset['data']
    f1 = exact_match = total = 0
    for article in dataset:
        for paragraph in article['paragraphs']:
            for qa in paragraph['qas']:
                total += 1
                if qa['id'] not in predictions:
                    continue
                
                ground_truths = list(map(lambda x: x['text'], qa['answers']))
                
                prediction = predictions[qa['id']]
                
                exact_match += metric_max_over_ground_truths(
                    exact_match_score, prediction, ground_truths)
                
                f1 += metric_max_over_ground_truths(
                    f1_score, prediction, ground_truths)
                
    
    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total
    
    return exact_match, f1



In [56]:
def normalize_answer(s):
    '''
    Performs a series of cleaning steps on the ground truth and 
    predicted answer.
    '''
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    '''
    Returns maximum value of metrics for predicition by model against
    multiple ground truths.
    
    :param func metric_fn: can be 'exact_match_score' or 'f1_score'
    :param str prediction: predicted answer span by the model
    :param list ground_truths: list of ground truths against which
                               metrics are calculated. Maximum values of 
                               metrics are chosen.
                            
    
    '''
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
        
    return max(scores_for_ground_truths)


def f1_score(prediction, ground_truth):
    '''
    Returns f1 score of two strings.
    '''
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    '''
    Returns exact_match_score of two strings.
    '''
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def epoch_time(start_time, end_time):
    '''
    Helper function to record epoch time.
    '''
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [57]:
if running_in_colab:
    from google.colab import drive
    drive.mount('/content/drive')

In [67]:
if load_train_epochs:
    checkpoint = torch.load(f"{model_name}.pth")
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

Due to the incredibly long training time of this model (more than 3 hours using GTX 1070!), the first three epochs was trained in another environment. Therefore, we can only show the output of the last 2 epochs here.

In [68]:

train_losses = []
valid_losses = []
ems = []
f1s = []
epochs = 5
best_valid_loss=99999

if running_in_colab:
    path=f'drive/MyDrive/{model_name}.pth'
else:
    path=f'{model_name}.pth'

for epoch in range(load_train_epochs,epochs):
    print(f"Epoch {epoch+1}")
    start_time = time.time()
    
    train_loss = train(model, train_dataset)
    valid_loss, em, f1 = valid(model, valid_dataset)
    
    
    if best_valid_loss>valid_loss: # save the best model
        torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': valid_loss,
                'em':em,
                'f1':f1,
                }, path)
        best_valid_loss=valid_loss

    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    ems.append(em)
    f1s.append(f1)

    print(f"Epoch train loss : {train_loss}| Time: {epoch_mins}m {epoch_secs}s")
    print(f"Epoch valid loss: {valid_loss}")
    print(f"Epoch EM: {em}")
    print(f"Epoch F1: {f1}")
    print("====================================================================================")
    

Epoch 4
Starting training ........
Starting batch: 0
Starting batch: 500
Starting batch: 1000
Starting batch: 1500
Starting batch: 2000
Starting batch: 2500
Starting batch: 3000
Starting batch: 3500
Starting batch: 4000
Starting batch: 4500
Starting batch: 5000
Starting batch: 5500
Starting batch: 6000
Starting batch: 6500
Starting batch: 7000
Starting batch: 7500
Starting batch: 8000
Starting batch: 8500
Starting batch: 9000
Starting batch: 9500
Starting batch: 10000
Starting batch: 10500
Starting validation .........
Starting batch 0
Starting batch 500
Starting batch 1000
Starting batch 1500
Starting batch 2000
Epoch train loss : 2.507835018201583| Time: 139m 19s
Epoch valid loss: 3.7418820450577335
Epoch EM: 55.950804162724694
Epoch F1: 67.76823864062513
Epoch 5
Starting training ........
Starting batch: 0
Starting batch: 500
Starting batch: 1000
Starting batch: 1500
Starting batch: 2000
Starting batch: 2500
Starting batch: 3000
Starting batch: 3500
Starting batch: 4000
Starting bat

## Visualizing Performance of different models

Note: there were no test dataset, only validation dataset was there. All the performance on the table was that after 3 epochs.

In [ ]:
from tabulate import tabulate

In [ ]:

overall_result = []
config_names=['AdaDelta_default','AdaDelta_default','AdaDelta_halved','Adam_default', 'SGD_default',
              'dropout-.15','dropout-.25','charwidth4','hidden50']

for i, _ in enumerate(config_names):
    indiv_result = []
    save_location =  f"results/{config_names[i]}.pth"
    checkpoint = torch.load(save_location)
    valid_loss=checkpoint['loss']
    em=checkpoint['em']  
    f1=checkpoint['f1']

    indiv_result.append(config_names[i])
    indiv_result.append(valid_loss)
    indiv_result.append(em)
    indiv_result.append(f1)
    overall_result.append(indiv_result)

print(tabulate(overall_result, headers=['Model', 'Valid Loss', 'EM', 'F1']))

## References
* Papers read/referred:
    1. BiDAF: https://arxiv.org/abs/1611.01603
    2. Convolutional Neural Networks for Sentence Classification: https://arxiv.org/abs/1408.5882
    3. Highway Networks: https://arxiv.org/abs/1505.00387
* Other helpful links:
    1. https://nlp.seas.harvard.edu/slides/aaai16.pdf. A great resource for character embeddings. The figures in the character embedding section are taken from here.
    2. https://towardsdatascience.com/the-definitive-guide-to-bi-directional-attention-flow-d0e96e9e666b. A great series of blogs to understand BiDAF.
    Some of the following repos might be out of date.
    3. https://github.com/allenai/bi-att-flow
    4. https://github.com/galsang
    5. https://github.com/jojonki/BiDAF/